In [2]:
from requests_html import HTMLSession
import requests
from bs4 import BeautifulSoup
import re
import time
import pandas as pd

# Crawl url

In [ ]:
def Oto_Crawler(startlink):
    Oto_link=startlink
    Link_list= list()
    session = HTMLSession()
    while True:
        
        req=session.get(Oto_link)
        element = req.html.html
        
        soup = BeautifulSoup(element,"lxml")
        product_list= soup.findAll("li",{"class":"car-item"})
        for product in product_list:
            link=product.find("a").get('href')
            Name=product.find("div",{"class":"cb2_02"}).text.strip()
            print(link)
            print(Name)
            Price=product.find("b",{"itemprop":"price"}).get("content")
            print(Price)
            Link_list.append({'OtoName':Name,'Price':Price,'Otolink':link})
        NEXT_PAGE_SELECTOR = soup.find("span",{"class":"pactive"}).text
        print(NEXT_PAGE_SELECTOR)
        
        NEXT_PAGE_SELECTOR=soup.find("span",{"url":f"https://bonbanh.com/oto-sf000000020/page,{int(NEXT_PAGE_SELECTOR)+1}"})
        
        if NEXT_PAGE_SELECTOR:
            Oto_link=NEXT_PAGE_SELECTOR.get('url')
            time.sleep(3)
            print(Oto_link)
        else:
            req.close()
            break
    return Link_list

In [ ]:
startlink='https://bonbanh.com/oto-sf000000020'
link=Oto_Crawler(startlink)

In [ ]:
print(link)

In [ ]:
df = pd.DataFrame.from_dict(link)
df.to_csv('otolink.csv', index=False,encoding="utf-8-sig",
                  columns=['Price', 'OtoName' , 'Otolink'])

# crawl chi tiết xe

In [102]:
carDetail=list()

In [103]:
def getdetail(price,name,url):
    Oto_link='https://bonbanh.com/'+ url
    Link_list= list()
    session = HTMLSession()
    req=session.get(Oto_link)
    element = req.html.html
        
    soup = BeautifulSoup(element,"lxml")
    #basic Info
    BasicInfo= soup.find("div",{"class":"col"})
    madeIn=BasicInfo.findAll("span",{"class":"inp"})[0].text
    status=BasicInfo.findAll("span",{"class":"inp"})[1].text
    carType=BasicInfo.findAll("span",{"class":"inp"})[2].text
    Km=BasicInfo.findAll("span",{"class":"inp"})[3].text
    Km=int(re.search(r'\d+', Km).group())
    carColor=BasicInfo.findAll("span",{"class":"inp"})[4].text
    carSeat=BasicInfo.findAll("span",{"class":"inp"})[7].text
    carSeat=int(re.search(r'\d+', carSeat).group())
    #fuel-engine
    BasicInfo= soup.findAll("div",{"class":"col"})[1]
    engine=BasicInfo.findAll("span",{"class":"inp"})[0].text.strip()
    gear=BasicInfo.findAll("span",{"class":"inp"})[2].text
    Wheeldrive=BasicInfo.findAll("span",{"class":"inp"})[3].text
    MPG=BasicInfo.findAll("span",{"class":"inp"})[4].text.strip()
    MPG=int(re.search(r'\d+', MPG).group())
    #print(MPG)
    time.sleep(2)
    carDetail.append({'Price':price,'Car':name,'madeIn':madeIn,'status':status,'carType':carType,'Km':Km,'carColor':carColor,
                     'carSeat':carSeat,'engine':engine,'gear':gear,'Wheeldrive':Wheeldrive})
    if len(carDetail)%10==0:
        f = open("log.txt", "w")
        f.write(url)
        f.write('\n')
        f.write(str(len(carDetail)))
        f.close()

In [104]:
#trong quá trình chạy có 1 số link không còn tồn tại nên phải đổi biến đầu vào df_loop
[getdetail(price,name,url) for price, name, url in zip(df_loop['Price'], df_loop['OtoName'],df_loop['Otolink'])]

KeyboardInterrupt: 

In [ ]:
print(len(carDetail))

In [54]:
carDetail

[{'Price': 582000000,
  'Car': 'Kia Cerato 2.0 AT - 2018',
  'madeIn': 'Lắp ráp trong nước',
  'status': 'Xe đã dùng',
  'carType': 'Sedan',
  'Km': 28,
  'carColor': 'Xanh',
  'carSeat': 5,
  'engine': 'Xăng\t2.0 L',
  'gear': 'Số tự động',
  'Wheeldrive': 'FWD - Dẫn động cầu trước'},
 {'Price': 920000000,
  'Car': 'Hyundai SantaFe 2.2L 4WD - 2018',
  'madeIn': 'Lắp ráp trong nước',
  'status': 'Xe đã dùng',
  'carType': 'SUV',
  'Km': 46,
  'carColor': 'Trắng',
  'carSeat': 7,
  'engine': 'Dầu\t2.2 L',
  'gear': 'Số tự động',
  'Wheeldrive': '4WD - Dẫn động 4 bánh'},
 {'Price': 675000000,
  'Car': 'Ford Focus Titanium 1.5L - 2018',
  'madeIn': 'Lắp ráp trong nước',
  'status': 'Xe đã dùng',
  'carType': 'Sedan',
  'Km': 10,
  'carColor': 'Trắng',
  'carSeat': 5,
  'engine': 'Xăng\t1.5 L',
  'gear': 'Số tự động',
  'Wheeldrive': 'FWD - Dẫn động cầu trước'},
 {'Price': 598000000,
  'Car': 'Mazda 3 1.5 AT - 2018',
  'madeIn': 'Lắp ráp trong nước',
  'status': 'Xe đã dùng',
  'carType': 

In [ ]:
data = pd.DataFrame.from_dict(carDetail)
data.to_csv('Data.csv', index=False,encoding="utf-8-sig",
                  columns=['Price', 'Car' , 'madeIn','status','carType','Km','carColor',
                     'carSeat','engine','gear','Wheeldrive'])

In [3]:
df = pd.read_csv("otolink.csv")
df.head()

,Price,OtoName,Otolink
0,764000000,Ford Ranger Limited 2.0L 4x4 AT - 2021,xe-ford-ranger-limited-2.0l-4x4-at-2021-3477761
1,1079000000,Kia Sorento Luxury 2.5 AT - 2021,xe-kia-sorento-luxury-2.5-at-2021-3461324
2,12000000000,Mercedes Benz G class G63 AMG - 2021,xe-mercedes_benz-g_class-g63-amg-2021-3435331
3,599000000,Ford Ranger XLS 2.2L 4x2 MT - 2021,xe-ford-ranger-xls-2.2l-4x2-mt-2021-2899786
4,750000000,Toyota Innova E 2.0 MT - 2021,xe-toyota-innova-e-2.0-mt-2021-3495966


# xữ lý các khi có link không còn tồn tại

In [99]:
# bỏ qua link bị hư xuất giá trị tiếp theo
df.loc[19842]

Price                                        535000000
OtoName             Ford Ranger XLS 2.2L 4x2 MT - 2017
Otolink    xe-ford-ranger-xls-2.2l-4x2-mt-2017-3557762
Name: 19842, dtype: object

In [100]:
# láy danh sách link tiếp theo sau đó chạy lại bước cào dữ liệu ở trên
df_loop=df.loc[19842:]

In [101]:
df_loop.head()

,Price,OtoName,Otolink
19842,535000000,Ford Ranger XLS 2.2L 4x2 MT - 2017,xe-ford-ranger-xls-2.2l-4x2-mt-2017-3557762
19843,276000000,Kia Morning Si MT - 2017,xe-kia-morning-si-mt-2017-3558089
19844,752000000,Ford Ranger Wildtrak 3.2L 4x4 AT - 2017,xe-ford-ranger-wildtrak-3.2l-4x4-at-2017-3470750
19845,789000000,Mazda CX5 2.0 AT - 2017,xe-mazda-cx5-2.0-at-2017-3567918
19846,3750000000,Mercedes Benz S class S500L - 2017,xe-mercedes_benz-s_class-s500l-2017-3567821


In [105]:
data = pd.DataFrame.from_dict(carDetail)

In [107]:
# ở đây ta chỉ lấy tầm 20000 dữ liệu nên data crawl được chỉ lấy tầm 170 dòng
dt_170=data.loc[:169]

In [110]:
# khi xuất hiện các dòng dữ liệu bị hư ta xuất dữ liệu ra file csv trước khi cho chạy lại
dt_170.to_csv('Data.csv', mode='a', header=False,index=False)

In [106]:
data


,Price,Car,madeIn,status,carType,Km,carColor,carSeat,engine,gear,Wheeldrive
0,535000000,Ford Ranger XLS 2.2L 4x2 MT - 2017,Nhập khẩu,Xe đã dùng,Bán tải / Pickup,76,Đỏ,5,Dầu\t2.2 L,Số tay,RFD - Dẫn động cầu sau
1,276000000,Kia Morning Si MT - 2017,Lắp ráp trong nước,Xe đã dùng,Hatchback,25,Đỏ,5,Xăng\t1.25 L,Số tay,FWD - Dẫn động cầu trước
2,752000000,Ford Ranger Wildtrak 3.2L 4x4 AT - 2017,Nhập khẩu,Xe đã dùng,Bán tải / Pickup,40,Vàng,5,Dầu\t3.2 L,Số tự động,4WD - Dẫn động 4 bánh
3,789000000,Mazda CX5 2.0 AT - 2017,Lắp ráp trong nước,Xe đã dùng,SUV,0,Xanh,5,Xăng\t2.0 L,Số tự động,FWD - Dẫn động cầu trước
4,3750000000,Mercedes Benz S class S500L - 2017,Lắp ráp trong nước,Xe đã dùng,Sedan,35,Đen,5,Xăng\t4.7 L,Số tự động,RFD - Dẫn động cầu sau
...,...,...,...,...,...,...,...,...,...,...,...
361,2890000000,Mercedes Benz S class S400L - 2017,Lắp ráp trong nước,Xe đã dùng,Sedan,40,Trắng,5,Xăng\t3.0 L,Số tự động,RFD - Dẫn động cầu sau
362,548000000,Mazda 3 1.5 AT - 2017,Lắp ráp trong nước,Xe đã dùng,Sedan,0,Trắng,5,Xăng\t1.5 L,Số tự động,FWD - Dẫn động cầu trước
363,543000000,Toyota Innova 2.0E - 2017,Lắp ráp trong nước,Xe đã dùng,Crossover,97,Xám,8,Xăng\t2.0 L,Số tay,RFD - Dẫn động cầu sau
364,528000000,Mazda 3 1.5 AT - 2017,Lắp ráp trong nước,Xe đã dùng,Hatchback,0,Trắng,5,Xăng\t1.5 L,Số tự động,FWD - Dẫn động cầu trước


In [108]:
dt_170

,Price,Car,madeIn,status,carType,Km,carColor,carSeat,engine,gear,Wheeldrive
0,535000000,Ford Ranger XLS 2.2L 4x2 MT - 2017,Nhập khẩu,Xe đã dùng,Bán tải / Pickup,76,Đỏ,5,Dầu\t2.2 L,Số tay,RFD - Dẫn động cầu sau
1,276000000,Kia Morning Si MT - 2017,Lắp ráp trong nước,Xe đã dùng,Hatchback,25,Đỏ,5,Xăng\t1.25 L,Số tay,FWD - Dẫn động cầu trước
2,752000000,Ford Ranger Wildtrak 3.2L 4x4 AT - 2017,Nhập khẩu,Xe đã dùng,Bán tải / Pickup,40,Vàng,5,Dầu\t3.2 L,Số tự động,4WD - Dẫn động 4 bánh
3,789000000,Mazda CX5 2.0 AT - 2017,Lắp ráp trong nước,Xe đã dùng,SUV,0,Xanh,5,Xăng\t2.0 L,Số tự động,FWD - Dẫn động cầu trước
4,3750000000,Mercedes Benz S class S500L - 2017,Lắp ráp trong nước,Xe đã dùng,Sedan,35,Đen,5,Xăng\t4.7 L,Số tự động,RFD - Dẫn động cầu sau
...,...,...,...,...,...,...,...,...,...,...,...
165,298000000,Kia Morning Si MT - 2017,Lắp ráp trong nước,Xe đã dùng,Hatchback,35,Đỏ,5,Xăng\t1.25 L,Số tay,FWD - Dẫn động cầu trước
166,475000000,Toyota Vios 1.5G - 2017,Lắp ráp trong nước,Xe đã dùng,Sedan,45,Cát,5,Xăng\t1.5 L,Số tự động,FWD - Dẫn động cầu trước
167,365000000,Chevrolet Cruze LT 1.6L - 2017,Lắp ráp trong nước,Xe đã dùng,Sedan,45,Ghi,5,Xăng\t1.6 L,Số tay,FWD - Dẫn động cầu trước
168,365000000,Chevrolet Cruze LT 1.6L - 2017,Lắp ráp trong nước,Xe đã dùng,Sedan,45,Trắng,5,Xăng\t1.6 L,Số tay,FWD - Dẫn động cầu trước
